In [1]:
import cv2

In [2]:
print ("Hello")

Hello


In [7]:
import urllib.request

In [4]:
import numpy as np

In [8]:
req = urllib.request.urlopen('http://www.hpca.ual.es/~vruiz/images/lena.png')

In [9]:
arr = np.asarray(bytearray(req.read()), dtype=np.uint8)

In [10]:
img = cv2.imdecode(arr,-1)

In [11]:
cv2.imshow('lalala',img)

error: /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window.cpp:545: error: (-2) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Carbon support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function cvShowImage
